# Project 2
### Antonia Sunseri 

In [25]:
# Loading Packages 
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio 
import dash
from dash import Dash, dcc, html, Input, Output
pio.templates.default = "plotly_white"

In [26]:
# Reading in dataframe
df = pd.read_csv(r'C:\Users\anton\OneDrive - University of Denver\Spring Quarter 2025\COMP 4433\country_vaccinations.csv')

# First 6 rows
df.head()


latest_df = df.sort_values('date').groupby('country').tail(1)

df_cleaned = df.dropna()

In [27]:
# Save to new dataframe
df_cleaned.to_csv("cleaned_country_vaccinations.csv", index=False)

In [28]:
app = dash.Dash(__name__)
app.title = "COVID-19 Global Vaccination Dashboard"

In [29]:
# App Layout
app.layout = html.Div([
    html.H1("Global COVID-19 Vaccination Dashboard", style={'textAlign': 'center'}),

    html.Div([
        html.Label("Select a Country:"),
        dcc.Dropdown(
            options=[{'label': c, 'value': c} for c in sorted(df['country'].unique())],
            value='United States',
            id='country-dropdown'
        ),

        html.Label("Select Metric:"),
        dcc.RadioItems(
            options=[
                {'label': 'People Fully Vaccinated', 'value': 'people_fully_vaccinated'},
                {'label': 'Daily Vaccinations', 'value': 'daily_vaccinations'}
            ],
            value='people_fully_vaccinated',
            id='metric-radio'
        ),

        html.Label("Select Date Range:"),
        dcc.DatePickerRange(
            id='date-range',
            min_date_allowed=df['date'].min(),
            max_date_allowed=df['date'].max(),
            start_date=df['date'].min(),
            end_date=df['date'].max()
        ),
    ], style={'width': '50%', 'margin': 'auto'}),

    dcc.Graph(id='line-graph'),

    html.H2("Top 10 Countries by Vaccination", style={'textAlign': 'center'}),
    dcc.Graph(
        id='bar-graph',
        figure=px.bar(
            latest_df.nlargest(10, 'people_fully_vaccinated'),
            x='country',
            y='people_fully_vaccinated',
            labels={'people_fully_vaccinated': 'Fully Vaccinated People'},
            title='Top 10 Countries (Latest Date)',
            template='plotly_white'
        )
    ),

    html.H2("Global Vaccination Map", style={'textAlign': 'center'}),
    dcc.Graph(
        id='map-graph',
        figure=px.choropleth(
            latest_df,
            locations='iso_code',
            color='people_fully_vaccinated',
            hover_name='country',
            color_continuous_scale='Viridis',
            title='Global Vaccinations (Latest Date)',
            template='plotly_white'
        )
    )
])


In [30]:
# Callback
@app.callback(
    Output('line-graph', 'figure'),
    Input('country-dropdown', 'value'),
    Input('metric-radio', 'value'),
    Input('date-range', 'start_date'),
    Input('date-range', 'end_date')
)
def update_line_chart(selected_country, selected_metric, start_date, end_date):
    filtered = df[
        (df['country'] == selected_country) &
        (df['date'] >= start_date) &
        (df['date'] <= end_date)
    ]
    fig = px.line(
        filtered,
        x='date',
        y=selected_metric,
        title=f"{selected_metric.replace('_', ' ').title()} in {selected_country}",
        labels={selected_metric: selected_metric.replace('_', ' ').title()},
        template='plotly_white'
    )
    return fig


In [24]:
if __name__ == '__main__':
    app.run(debug=True)